November 4, 2021

Burton Rosenberg

Descrete logs and easy bits for the Diffie-Hellman problem

In [1]:
"""
General number theory computations
"""


def exp_mod(a,b,p):
    
    a = a%p
    b = b%(p-1)

    if a==0:
        return 0
    if a==1:
        return 1
    if b==0:
        return 1
    if b==1:
        return a
    assert b>1
    
    if b%2==1:
        return a*exp_mod(a,b-1,p)%p
    t = exp_mod(a,b//2,p)
    return (t*t)%p

def qr_p(a,p):
    return exp_mod(a,(p-1)//2,p)==1

def extended_gcd(a,b):
    """
    extended GCD algorithm. recursive.
    returns (d,s,t) where d = s*a+t*b 
    and d = gcd(a,b)
    """
    assert(
        a>=0 and b>=0 )
    if b==0:
        return (a,1,0)
    (q,r) = divmod(a,b)
    (d,s,t) = extended_gcd(b,r)
    # gcd(a, b) == gcd(b, r) == s*b + t*r == s*b + t*(a - q*b)
    return (d,t,s-q*t)

def invert(a,p):
    (d,t,s) = extended_gcd(a,p)
    assert 1==d
    return t%p
    
def gen_p(g,p,verbose=False):
    r = g 
    for i in range(1,p-1):
        if verbose: print (f'{i}:\t{r}')
        if r==1:
            return False
        r = (r*g)%p
    if verbose: print (f'{p-1}:\t{r}')
    return True

def find_gen(p):
    for g in range(2,p-1):
        if gen_p(g,p):
            return g
    assert False

def brute_log(x,g,p):
    x = x%p
    assert x!=0
    
    r = g
    if x==1:
        return 0
    for i in range(1,p-1):
        if r==x:
            return i
        r = r*g%p
    assert False

  
    

In [2]:
class TonelliShanks:
    
    def __init__(self,p):
        self.p = p

    def find_qnr(self):
        for i in range(2,self.p-1):
            if not qr_p(i,self.p):
                return i
        assert False

    def make_one(self,t):
        i = 1
        e = (t*t)%self.p
        while e!=1:
            i += 1
            e = (e*e)%self.p
        return i

    @staticmethod
    def s_q_form(n):
        i = 0
        while n%2==0:
            i += 1
            n //= 2
        return (i,n)

    def sq_root(self,n):
        if not qr_p(n,self.p):
            return 0 

        (s,q) = TonelliShanks.s_q_form(self.p-1)
        z = self.find_qnr()
        m = s
        c = exp_mod(z,q,self.p)
        t = exp_mod(n,q,self.p)
        r = exp_mod(n,((q+1)//2),self.p)
        while True:
            if t==0: return 0
            if t==1: return r
            i = self.make_one(t)
            b = exp_mod(c,2**(m-i-1),self.p)
            m = i
            c = exp_mod(b,2,self.p)
            t = (t*c)%self.p
            r = (r*b)%self.p
        assert False
 
class EasyBits:
    
    def __init__(self,p,g):
        self.p = p
        self.g = g
        self.inv_g = invert(g,p)
        (s,t) = TonelliShanks.s_q_form(p-1)
        self.ts = TonelliShanks(p)
        self.s = s
        self.t = t

    def how_many(self):
        print(f'there are {self.s} easy bits')
        
    def easy_bits(self,n):
        eb = []
        for i in range(self.s):
            if qr_p(n,self.p):
                eb = [0]+eb
            else:
                eb = [1]+eb
                n = n*self.inv_g % self.p
            n = self.ts.sq_root(n)
        return eb
            
      

In [3]:
my_prime = 11
my_gen = find_gen(my_prime)
print(f'<{my_gen}> = Z/{my_prime}Z')
gen_p(my_gen,my_prime,verbose=True)

tonelli_shanks = TonelliShanks(my_prime)
for i in range(1,my_prime):
    sq = tonelli_shanks.sq_root(i)
    if sq!=0:
        assert (sq*sq)%my_prime==i
        print(f'sqrt({i})=\t{sq}, {(my_prime-sq%my_prime)}')

<2> = Z/11Z
1:	2
2:	4
3:	8
4:	5
5:	10
6:	9
7:	7
8:	3
9:	6
10:	1
sqrt(1)=	1, 10
sqrt(3)=	5, 6
sqrt(4)=	9, 2
sqrt(5)=	4, 7
sqrt(9)=	3, 8


In [5]:
      
my_prime = 97
my_gen = find_gen(my_prime)        
eb = EasyBits(my_prime,my_gen)

gen_p(my_gen,my_prime,verbose=True)

n = 47
print(eb.easy_bits(n))
dl = brute_log(n,my_gen,my_prime)
print(bin(dl))

1:	5
2:	25
3:	28
4:	43
5:	21
6:	8
7:	40
8:	6
9:	30
10:	53
11:	71
12:	64
13:	29
14:	48
15:	46
16:	36
17:	83
18:	27
19:	38
20:	93
21:	77
22:	94
23:	82
24:	22
25:	13
26:	65
27:	34
28:	73
29:	74
30:	79
31:	7
32:	35
33:	78
34:	2
35:	10
36:	50
37:	56
38:	86
39:	42
40:	16
41:	80
42:	12
43:	60
44:	9
45:	45
46:	31
47:	58
48:	96
49:	92
50:	72
51:	69
52:	54
53:	76
54:	89
55:	57
56:	91
57:	67
58:	44
59:	26
60:	33
61:	68
62:	49
63:	51
64:	61
65:	14
66:	70
67:	59
68:	4
69:	20
70:	3
71:	15
72:	75
73:	84
74:	32
75:	63
76:	24
77:	23
78:	18
79:	90
80:	62
81:	19
82:	95
83:	87
84:	47
85:	41
86:	11
87:	55
88:	81
89:	17
90:	85
91:	37
92:	88
93:	52
94:	66
95:	39
96:	1
[1, 0, 1, 0, 0]
0b1010100
